# Real-Time API
Using the real-time API you can access all data generated by your Pupil Invisible devices in real-time. Further, you can control all devices remotely to start and stop recordings or save events.
The API allows you to do this from within your own custom scripts. If your are simply looking for a tool to monitor and controll all your devices in real-time, check out [Pupil Invisible Monitor]().

To use the API the Companion devices and the computer accessing the API need to be connected to the same local network. We have created a Python client for the API that makes it very easy to use. If you require access to the API from a different programming language

In [ ]:
from pupil_labs.realtime_api.basic import discover_one_device
device = discover_one_device(max_search_duration_seconds=10)

If no device can be found -> Troubleshooting

In [ ]:
print(f"Phone IP address: {device.phone_ip}")
print(f"Phone name: {device.phone_name}")
print(f"Phone unique ID: {device.phone_id}")

print(f"Battery level: {device.battery_level_percent}%")
print(f"Battery state: {device.battery_state}")

print(f"Free storage: {device.memory_num_free_bytes / 1024**3}GB")
print(f"Storage level: {device.memory_state}")

print(f"Connected glasses: SN {device.serial_number_glasses}")
print(f"Connected scene camera: SN {device.serial_number_scene_cam}")

In [ ]:
import time
print(f"Starting recording")
recording_id = device.recording_start()
print(f"Started recording with id {recording_id}")

time.sleep(5)
status = device.get_status()

print(f"Recording is running for {status.recording.rec_duration_seconds} seconds")
device.recording_stop_and_save()

print("Recording stopped and saved")

In [ ]:
print(device.send_event("test event"))

# send event with current timestamp
print(device.send_event("test event", event_timestamp_unix_ns=time.time_ns()))

In [ ]:
frame = device.read_scene_video_frame()
bgr_buffer = frame.bgr_buffer()

import matplotlib.pyplot as plt

plt.imshow(bgr_buffer)